# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
# !nvidia-smi

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [2]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# print(teacher.summary())

In [3]:

# Train the teacher model as usual
epochs = 12
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


# #checking teachers test accuracy
# X_test = X_test.reshape(10000, 28, 28, 1)

# X_test = X_test.astype('float32')
# # Normalize the values
# X_test /= 255
# preds=teacher.predict(X_test)

# numbers=[0,1,2,3,4,5,6,7,8,9]
# preds2=[]
# for p in preds:
#     preds2.append(numbers[np.argmax(p)])
# preds2=np.array(preds2)

# print (y_test)
# print (preds2)
# print (np.sum(y_test==preds2))

Epoch 1/12
188/188 [==============================] - 4s 14ms/step - loss: 0.6123 - accuracy: 0.8004 - val_loss: 0.0700 - val_accuracy: 0.9800
Epoch 2/12
188/188 [==============================] - 2s 13ms/step - loss: 0.0814 - accuracy: 0.9761 - val_loss: 0.0588 - val_accuracy: 0.9823
Epoch 3/12
188/188 [==============================] - 2s 12ms/step - loss: 0.0456 - accuracy: 0.9849 - val_loss: 0.0511 - val_accuracy: 0.9840
Epoch 4/12
188/188 [==============================] - 2s 12ms/step - loss: 0.0354 - accuracy: 0.9885 - val_loss: 0.0476 - val_accuracy: 0.9862
Epoch 5/12
188/188 [==============================] - 2s 12ms/step - loss: 0.0246 - accuracy: 0.9923 - val_loss: 0.0440 - val_accuracy: 0.9876
Epoch 6/12
188/188 [==============================] - 2s 12ms/step - loss: 0.0185 - accuracy: 0.9943 - val_loss: 0.0429 - val_accuracy: 0.9893
Epoch 7/12
188/188 [==============================] - 2s 13ms/step - loss: 0.0167 - accuracy: 0.9948 - val_loss: 0.0432 - val_accuracy: 0.9900

### Evaluation 

In [4]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [4]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [5]:
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
# Normalize the values
X_test /= 255

In [6]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0359 - accuracy: 0.9901


[0.03592796251177788, 0.9901000261306763]

### Dense Vector split

In [7]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)


# 2 Students case
# ---------------------------------------------
s1Train=train_dense[:,:128]
s2Train=train_dense[:,128:]

## GANs' Training

In [8]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

### Discriminator

In [9]:
def build_sdiscriminator():
    
    input2 = Input(shape=(128,),name='input')
    inp=Dense(64)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)

    output2=Dense(128)(leaky_relu)

    
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])


    return disc

In [10]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [28]:
# d1= build_sdiscriminator()
# d1.summary()

### Generator

In [54]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (5, 5), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(32, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(128,name='req'+name))

    student1.compile(opt,loss='mean_squared_error',metrics=['accuracy'])
    
    return student1


build_sgenerator('s').summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s (Conv2D)                   (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 20, 20, 32)        25632     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 8, 8, 16)          4624      
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 6, 6, 16)          2320      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 3, 3, 16)          0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 3, 3, 16)        

In [55]:
def training(generator,discriminator,gan,features,epo=20):
    # Setup Models here
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            # Split
            inputs=X_train[index:index+BATCH_SIZE]
            sXtrain = features[index:index+BATCH_SIZE]

            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(sXtrain,[y_real,sXtrain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,sXtrain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,sXtrain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

You can initialize multiple instances here 

In [56]:
discriminator1 = build_sdiscriminator()
discriminator2 = build_sdiscriminator()

In [57]:
s1=build_sgenerator("s1")
s2=build_sgenerator('s2')

In [58]:
gan1 = build_gan(s1,discriminator1)
gan2 = build_gan(s2,discriminator2)

In [59]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=40)
s2 = training(s2,discriminator2,gan2,s2Train,epo=45)

375/375 [==============================] - 21s 51ms/step
Epoch: 1 | Discriminator Loss: 0.417165 | Generator Loss: 1.400226 | 
375/375 [==============================] - 19s 49ms/step
Epoch: 2 | Discriminator Loss: 0.204496 | Generator Loss: 0.874988 | 
375/375 [==============================] - 19s 50ms/step
Epoch: 3 | Discriminator Loss: 0.174200 | Generator Loss: 0.777061 | 
375/375 [==============================] - 18s 48ms/step
Epoch: 4 | Discriminator Loss: 0.159355 | Generator Loss: 0.722006 | 
375/375 [==============================] - 19s 50ms/step
Epoch: 5 | Discriminator Loss: 0.150087 | Generator Loss: 0.694982 | 
375/375 [==============================] - 19s 50ms/step
Epoch: 6 | Discriminator Loss: 0.143383 | Generator Loss: 0.684482 | 
375/375 [==============================] - 18s 49ms/step
Epoch: 7 | Discriminator Loss: 0.137977 | Generator Loss: 0.657637 | 
375/375 [==============================] - 18s 49ms/step
Epoch: 8 | Discriminator Loss: 0.132203 | Generator Lo

375/375 [==============================] - 17s 46ms/step
Epoch: 25 | Discriminator Loss: 0.093059 | Generator Loss: 0.400944 | 
375/375 [==============================] - 17s 46ms/step
Epoch: 26 | Discriminator Loss: 0.091846 | Generator Loss: 0.397737 | 
375/375 [==============================] - 17s 44ms/step
Epoch: 27 | Discriminator Loss: 0.091650 | Generator Loss: 0.393145 | 
375/375 [==============================] - 18s 47ms/step
Epoch: 28 | Discriminator Loss: 0.091019 | Generator Loss: 0.388581 | 
375/375 [==============================] - 18s 47ms/step
Epoch: 29 | Discriminator Loss: 0.090506 | Generator Loss: 0.383971 | 
375/375 [==============================] - 17s 46ms/step
Epoch: 30 | Discriminator Loss: 0.089556 | Generator Loss: 0.381582 | 
375/375 [==============================] - 17s 46ms/step
Epoch: 31 | Discriminator Loss: 0.088351 | Generator Loss: 0.380525 | 
375/375 [==============================] - 17s 46ms/step
Epoch: 32 | Discriminator Loss: 0.087938 | Gene

## **2 Students**

In [60]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
output=tensorflow.keras.layers.concatenate([o1,o2])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm2=Model([s1.get_layer("s1").input,s2.get_layer("s2").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm2.get_layer('d1').set_weights(my_weights)

In [61]:
i=0
for l in mm2.layers[:len(mm2.layers)-1]:
    l.trainable=False

mm2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [62]:
batch_size = 128
mm2_history=mm2.fit([X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1, validation_data=([X_val,X_val], Y_val))

Epoch 1/20
375/375 [==============================] - 4s 9ms/step - loss: 1.4530 - accuracy: 0.7553 - val_loss: 0.0764 - val_accuracy: 0.9851
Epoch 2/20
375/375 [==============================] - 3s 8ms/step - loss: 0.1714 - accuracy: 0.9616 - val_loss: 0.0750 - val_accuracy: 0.9858
Epoch 3/20
375/375 [==============================] - 4s 10ms/step - loss: 0.1395 - accuracy: 0.9687 - val_loss: 0.0710 - val_accuracy: 0.9863
Epoch 4/20
375/375 [==============================] - 4s 10ms/step - loss: 0.1200 - accuracy: 0.9724 - val_loss: 0.0651 - val_accuracy: 0.9865
Epoch 5/20
375/375 [==============================] - 4s 10ms/step - loss: 0.1025 - accuracy: 0.9739 - val_loss: 0.0617 - val_accuracy: 0.9864
Epoch 6/20
375/375 [==============================] - 3s 9ms/step - loss: 0.0989 - accuracy: 0.9751 - val_loss: 0.0598 - val_accuracy: 0.9867
Epoch 7/20
375/375 [==============================] - 3s 8ms/step - loss: 0.0958 - accuracy: 0.9774 - val_loss: 0.0560 - val_accuracy: 0.9866
Epo

In [63]:
loss, acc = mm2.evaluate([X_test,X_test], Y_test, verbose=1)
loss, acc 

313/313 [==============================] - 2s 6ms/step - loss: 0.0360 - accuracy: 0.9896


(0.03601359203457832, 0.9896000027656555)

In [64]:
mm2.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s1 (Conv2D)                     (None, 24, 24, 32)   832         s1_input[0][0]                   
__________________________________________________________________________________________________
s2 (Conv2D)                     (None, 24, 24, 32)   832         s2_input[0][0]                   
___________________________________________________________________________________________